# Regression Model A: Field 5

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 5 (F5)
Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E5" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E5" ),]
dE4=d4[which(d4$Estate=="E5" ),]
dE5=d5[which(d5$Estate=="E5" ),]

## Variables

Declare the variable and construct new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL
foliar=d2$Diff
prun.time=dE3$Frequency
prun.cost=dE3$PruningCost
manuring.time=dE4$Frequency
manuring.cost=dE4$ManuringCost
pd.time=dE5$Frequency
pd.cost=dE5$PDCost

dg1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Correlation

Check the correlations for all variables. 

In [6]:
c1=cor(dg1[,-1:-2])

corrplot(c1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(c1, type="upper")

## Regression

Fit the data using multiple regression for all variables. 

In [7]:
a1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=dg1)
summary(a1)
plot(a1,which=1)
plot(a1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = dg1)

Residuals:
    Min      1Q  Median      3Q     Max 
-68.930 -19.062   2.137  20.107  68.949 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.411e+02  1.757e+01   8.032 1.87e-09 ***
rainfall       1.518e-02  5.428e-02   0.280   0.7813    
foliar        -4.305e+00  1.667e+00  -2.581   0.0142 *  
prun.time     -3.490e+00  6.288e+00  -0.555   0.5824    
prun.cost      4.667e-03  8.008e-03   0.583   0.5638    
manuring.time  7.431e+00  6.522e+00   1.139   0.2623    
manuring.cost  3.140e-05  3.204e-04   0.098   0.9225    
pd.time       -8.818e+00  7.255e+00  -1.215   0.2323    
pd.cost        3.626e-03  5.057e-03   0.717   0.4782    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 32.7 on 35 degrees of freedom
Multiple R-squared:  0.2947,	Adjusted R-squared:  0.1335 


## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 


The stepwise method incorporate with VIF factor is better than anova method in order to     select the significant variables.    

### Using ANOVA

In [8]:
anova(a1)

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
rainfall,1,5.348645e-01,5.348645e-01,5.003081e-04,0.982281788
foliar,1,1.128592e+04,1.128592e+04,1.055676e+01,0.002558298
prun.time,1,1.898387e+02,1.898387e+02,1.775737e-01,0.676043531
prun.cost,1,1.543379e+02,1.543379e+02,1.443664e-01,0.706274169
manuring.time,1,2.417145e+03,2.417145e+03,2.260979e+00,0.141640932
manuring.cost,1,5.343620e+00,5.343620e+00,4.998381e-03,0.944039567
pd.time,1,1.029824e+03,1.029824e+03,9.632895e-01,0.333095589
pd.cost,1,5.494593e+02,5.494593e+02,5.139600e-01,0.478186829
Residuals,35,3.741746e+04,1.069070e+03,NA,NA


### Using stepwise

In [9]:
selectedMod <- step(a1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

Start:  AIC=314.81
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    manuring.cost + pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- manuring.cost  1      10.3 37428 312.82
- rainfall       1      83.7 37501 312.91
- prun.time      1     329.4 37747 313.20
- prun.cost      1     363.1 37781 313.24
- pd.cost        1     549.5 37967 313.45
- manuring.time  1    1388.0 38805 314.41
- pd.time        1    1579.3 38997 314.63
<none>                       37417 314.81
- foliar         1    7124.1 44542 320.48

Step:  AIC=312.82
yield ~ rainfall + foliar + prun.time + prun.cost + manuring.time + 
    pd.time + pd.cost

                Df Sum of Sq   RSS    AIC
- rainfall       1      91.1 37519 310.93
- prun.time      1     329.0 37757 311.21
- prun.cost      1     356.2 37784 311.24
- pd.cost        1     540.3 37968 311.45
- pd.time        1    1574.3 39002 312.64
<none>                       37428 312.82
- manuring.time  1    2720.3 40148 313.91
- foli


Call:
lm(formula = yield ~ foliar + manuring.time, data = dg1)

Residuals:
    Min      1Q  Median      3Q     Max 
-79.753 -19.598   5.267  21.450  67.933 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)    136.034     10.167  13.380  < 2e-16 ***
foliar          -4.684      1.396  -3.355  0.00172 ** 
manuring.time    7.441      4.665   1.595  0.11834    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 31.13 on 41 degrees of freedom
Multiple R-squared:  0.2509,	Adjusted R-squared:  0.2144 
F-statistic: 6.867 on 2 and 41 DF,  p-value: 0.002679


       foliar manuring.time 
     1.000037      1.000037 



Call:
lm(formula = yield ~ foliar + manuring.time, data = dg1)

Residuals:
    Min      1Q  Median      3Q     Max 
-79.753 -19.598   5.267  21.450  67.933 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)    136.034     10.167  13.380  < 2e-16 ***
foliar          -4.684      1.396  -3.355  0.00172 ** 
manuring.time    7.441      4.665   1.595  0.11834    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 31.13 on 41 degrees of freedom
Multiple R-squared:  0.2509,	Adjusted R-squared:  0.2144 
F-statistic: 6.867 on 2 and 41 DF,  p-value: 0.002679


## Prediction

Predict with the resample values.

In [10]:
set.seed(123)
dg3=sample(1:nrow(dg1),12)
dg4=dg1[dg3,]
estimate=predict(selectedMod,newdata=dg4)
estimate
sum.estimate=sum(estimate)
dg5=data.frame(estimate,sum.estimate)

13        34        18        37        38         2        21        43 
136.03431 101.32054 127.49740 143.47559 131.35041 153.67425  98.56315 130.25478 
       20        16        33        15 
103.24705 129.42390 120.88699 134.10780